In [36]:
import numpy as np
import uproot

In [39]:
def split_list(lst):
    """Divide a list in chunck_size sublists"""
    chuck_size = 2
    return [lst[i:i + chuck_size] for i in range(0, len(lst), chuck_size)]

In [40]:
lista = [1,2,3,4,5,6,7,8,9]
split_list(lista)

[[1, 2], [3, 4], [5, 6], [7, 8], [9]]

In [30]:
directory = 'C:/snoplus data/solarnu_analysis/mc/solar_nu/split_simu_Analysis_solar_Nue.root'

file = uproot.open(directory)
file.keys()

data = file['T;4']
pmt_info = file['pmt;2']

In [32]:
type(pmt_info)

uproot.models.TTree.Model_TTree_v20

In [20]:
pmt_info.keys()

['pmt_id', 'pmt_pos_xyz', 'pmt_pos_sph', 'pmt_type']

In [22]:
pmt_pos = np.array(pmt_info['pmt_pos_xyz'])[0]
vec = np.array([1,1,1])

array([-99999., -99999., -99999.])

# SunDirection Function

In [3]:
import numpy as np

def sun_direction(UTdays, UTsecs, UTnsecs):

    #Esta funcion no recibe un array de numeros! solo valores numericos y no listas.
    # Constantes
    labtwist_degrees = -49.58
    pi = np.pi
    twopi = 2 * pi
    to_rad = pi / 180.0

    # Offset en días: diferencia entre el tiempo cero de SNO+ (1 Jan 2010) y Jan 0, 2000
    days_offset = 2451543.5 - 2455197.5

    # Días desde Jan 0, 2000
    days = UTdays + UTsecs / 86400.0 + UTnsecs / (1.0e9 * 86400.0) - days_offset

    # Constantes astronómicas
    ecl = (23.4393 - 3.563E-7 * days) * to_rad
    w = (282.9404 + 4.70935E-5 * days) * to_rad
    e = 0.016709 - 1.151E-9 * days
    ma = (356.0470 + 0.9856002585 * days) * to_rad

    # Eccentric anomaly
    ea = ma + e * np.sin(ma) * (1.0 + e * np.cos(ma))

    # Dirección hacia el Sol (coordenadas eclípticas)
    xv = np.cos(ea) - e
    yv = np.sqrt(1.0 - e * e) * np.sin(ea)

    # True anomaly
    v = np.arctan2(yv, xv)

    # Longitud solar
    sun_longitude = v + w

    # Coordenadas geocéntricas eclípticas
    xs = np.cos(sun_longitude)
    ys = np.sin(sun_longitude)
    vec1 = np.array([
        ys * np.cos(ecl),
        ys * np.sin(ecl),
        xs
    ])

    # Rotación terrestre diaria
    k0 = 0.27499
    k1 = 1.0 + 1.0 / 365.2425
    spin = k0 + k1 * days
    spin_angle = (spin - int(spin)) * twopi

    # Latitud y longitud del detector (en radianes)
    longitude = (81 + 12 / 60 + 5 / 3600) * to_rad
    latitude = (46 + 28 / 60 + 31.0 / 3600) * to_rad
    labtwist = labtwist_degrees * to_rad

    # Rotaciones: Z(labtwist) * X(latitude) * Y(longitude - spin_angle)
    def rotation_matrix_y(angle):
        return np.array([
            [np.cos(angle), 0, np.sin(angle)],
            [0, 1, 0],
            [-np.sin(angle), 0, np.cos(angle)]
        ])

    def rotation_matrix_x(angle):
        return np.array([
            [1, 0, 0],
            [0, np.cos(angle), -np.sin(angle)],
            [0, np.sin(angle), np.cos(angle)]
        ])

    def rotation_matrix_z(angle):
        return np.array([
            [np.cos(angle), -np.sin(angle), 0],
            [np.sin(angle), np.cos(angle), 0],
            [0, 0, 1]
        ])

    rot = (
        rotation_matrix_z(labtwist) @
        rotation_matrix_x(latitude) @
        rotation_matrix_y(longitude - spin_angle)
    )

    sun_dir = rot @ vec1  # Producto matriz * vector

    return sun_dir  # Es un array de 3 elementos: [x, y, z]

In [27]:
UTdays = np.array([1,1,1])
UTsecs = np.array([1,1,1])
UTnsecs = np.array([1,1,1])

np.linalg.norm(sun_direction(UTdays = 100, UTsecs = 0.1, UTnsecs = 50))

1.0

# Build function to Extract the Observables for Solarnu Analysis

In [59]:
import numpy as np
import uproot

#Function to Compute the radial position of events
def magnitude(vector): 
    x = vector[:,0]
    y = vector[:,1]
    z = vector[:,2]

    r = np.sqrt(x**2 + y**2 + z**2)
    r = r.astype(np.float32)
    return r

#Analysis code settings  ------------------------------
#Read files:
input_file_dir = '' #directory and name to read the input file
out_file_dir = '' #directory to save the output files

#Data Cuts:
energy_inf_cut = 2.5
energy_sup_cut = 12

time_res_inf_cut = -5.0
time_res_sup_cut = 7.0

posr_cut = 5500

dcflag_cut = int(0x2100000042C2)

#Nº of splitten loops to analyze the data: 
#N_split = 20 # the more split, the less memory expensive is the code
#------------------------------------------------------

#Load the Data:
load_data = uproot.open(input_file_dir)

#select the tree of event data and PMT info
event_data = load_data['T;4']
pmt_data = load_data['pmt;2']

#event info to be used:
var_event_list = ['evtid', 'energy', 'position', 'hit_pmtid', 
                  'hit_residual', 'dc_flag', 'ev_time_day', 
                  'ev_time_sec', 'ev_time_nanosec']  #list the name of the varibles to be extracted and used for the solarnu analysis.

#pmt info to be used
var_pmt_list = ['pmt_id', 'pmt_pos_xyz', 'pmt_type']

#Observables to save
var_name_save_list = ['energy', 'posr', 'hit_residual']
multi_cos_alpha = np.array([]) #create the empty list of the cos_alpha for the multiple PMTs record

# Extract the variables with the name of the var_event_list in numpy.array from the .root file
for var_name_i in var_event_list:
    locals()[var_name_i] = np.array(event_data[var_name_i])
    print(f'Loaded Observable {var_name_i} and value: ', locals()[var_name_i])

# Compute the distance from the reconst. vertx to the center of the detector
posr = magnitude(position)

# Extract the pmt info
for var_name_i in var_pmt_list:
    locals()[var_name_i] = np.array(pmt_data[var_name_i])
    print(f'Loaded PMT infor {var_name_i} and values: ', locals()[var_name_i])

# Select the valid PMT information through pmt_type selection
pmt_type_condition = (pmt_type == 1)

#Selecting the pmt_id_valid is enough to filter all the PMT info (as the pmt_pos):
#for example, selecting the valid hit_pmtid will automatically choose the valid pmt_pos using the pmtid as the index of the data
pmt_id_valid = pmt_id[pmt_type_condition] #list with the valid pmt_id

#Extract the valid components of the event data through the selection of the pmt_id_valid:
hit_pmt_id_condition = np.in1d(hit_pmtid, pmt_id_valid)
#Also include  general cuts on energy and event position, and a dcFlag cut
energy_condition = (energy >= energy_inf_cut) & (energy <= energy_sup_cut)
time_res_condition = (hit_residual >= time_res_inf_cut) & (hit_residual <= time_res_sup_cut)
posr_condition = (posr <= 5500 )
dcFlag_condition = ((dcflag_cut & dc_flag) == int(mask_cut))

general_condition = hit_pmt_id_condition & energy_condition & time_res_condition & posr_condition & dcFlag_condition

#Loop to extract the samples which verify general condition
for var_name_i in var_event_list:
    #print('before pmt_id selection', locals()[var_name_i].shape)
    locals()[var_name_i] = locals()[var_name_i][general_condition]  #selection of the observable entries that verify the general condition
    #print('after pmt_id selection', locals()[var_name_i].shape)
posr = posr[general_condition]

#Now, save the observables of interest from var_event_list after general condition cut. This include the Time Residuals!
for var_name_i in var_name_save_list:
    np.save(out_file_dir + var_name_i, locals()[var_name_i])

#Loop over PMTs records to compute the angle between each hit PMT and the Sun direction:
N_samples = len(hit_residual) #Number of samples to be analyzed
#sample_idx_split = np.slipt(np.array(range(N_samples)), N_split) # A list splitten in N_sample arrays which contains the sample index

for sample_idx in range(N_samples):

    #Extract universal time for this event:
    Utime_day = ev_time_day[sample_idx]
    Utime_sec = ev_time_sec[sample_idx]
    Utime_nsec = ev_time_nanosec[sample_idx]

    #Compute the Sun's outward direction
    sun_dir = sun_direction(UTdays = Utime_day, UTsecs = Utime_sec, UTnsecs = Utime_nsec)*(-1)
    print(f'sun direction: {sun_dir}')

    #Extract the hit PMT coordinates:
    pmt_hit_id = hit_pmtid[sample_idx]  # first take the id of the hit PMT to look for its coordinates. 
    pmt_hit_xyz = pmt_pos_xyz[pmt_hit_id] # The ID is equivalent to the index of the pmt_pos.
    print(f'pmt direction: {pmt_hit_xyz}')

    #Compute the cos_alpha of the angles between hit_pmt with the sun direction by the scalar product definition: The Sun vector verifies norm = 1
    #cos(alpha) = dot_prod/norm1*norm2  
    norm2 = np.linalg.norm(pmt_hit_xyz)
    pmt_hit_xyz = pmt_hit_xyz/norm2  #normalized vector
    
    dot_prod = np.dot(sun_dir, pmt_hit_xyz)
    
    cos_alpha = dot_prod
    print(f'cos of angle = {cos_alpha}')
    
    multi_cos_alpha = np.append(multi_cos_alpha, cos_alpha)

    #save the cos_alpha computation
    np.save(out_file_dir + 'cos_alpha', multi_cos_alpha)
